In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Thyroid_Cancer/GSE151180'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and miRNA expression in radioiodine refractory and avid papillary thyroid carcinomas (miRNA expresseion dataset)"
!Series_summary	"We performed gene and miRNA expression profiling in a series of 39 papillary thyroid carcinomas (PTCs) and 13 matched non-neoplastic thyroids derived from PTC patients with metastatic disease and submitted to radioiodine (RAI) treatment."
!Series_overall_design	"Gene and miRNA expression profiles were established by microarray analysis in a retrospective series of 52 snap-frozen thyroid samples including 35 tissues collected before RAI treatment (17 primary PTC tumors, 5 synchronous lymph node metastases (LNMs), and 13 matched non-neoplastic thyroids included as control) and 17 RAI-refractory LNMs collected as successive surgery following RAI treatment. Patients were stratified based on RAI uptake at the metastatic site and on RAI response in either avid or refractory, displaying disease remission or persistance, 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 4  # Patient RAI response
age_row = None  # Age is not available
gender_row = None  # Gender is not available

def convert_trait(value):
    if 'Avid' in value:
        return 1
    elif 'Refractory' in value:
        return 0
    return None

def convert_age(value):
    return None

def convert_gender(value):
    return None

# Save cohort info
save_cohort_info('GSE151180', './preprocessed/Thyroid_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Thyroid_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Thyroid_Cancer/trait_data/GSE151180.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4567964': [1], 'GSM4567965': [1], 'GSM4567966': [1], 'GSM4567967': [1], 'GSM4567968': [0], 'GSM4567969': [0], 'GSM4567970': [0], 'GSM4567971': [0], 'GSM4567972': [0], 'GSM4567973': [0], 'GSM4567974': [0], 'GSM4567975': [0], 'GSM4567976': [0], 'GSM4567977': [0], 'GSM4567978': [0], 'GSM4567979': [0], 'GSM4567980': [0], 'GSM4567981': [0], 'GSM4567982': [0], 'GSM4567983': [0], 'GSM4567984': [0], 'GSM4567985': [0], 'GSM4567986': [0], 'GSM4567987': [0], 'GSM4567988': [0], 'GSM4567989': [0], 'GSM4567990': [0], 'GSM4567991': [0], 'GSM4567992': [0], 'GSM4567993': [0], 'GSM4567994': [0], 'GSM4567995': [0], 'GSM4567996': [0], 'GSM4567997': [0], 'GSM4567998': [0], 'GSM4567999': [0], 'GSM4568000': [1], 'GSM4568001': [1], 'GSM4568002': [0], 'GSM4568003': [0], 'GSM4568004': [0], 'GSM4568005': [0], 'GSM4568006': [0], 'GSM4568007': [0], 'GSM4568008': [0], 'GSM4568009': [0], 'GSM4568010': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14',
       '15', '16', '17', '18', '21', '23', '24'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'SPOT_ID': ['miRNABrightCorner30', 'Blank', 'Blank', 'Blank', 'Blank'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'miRNA_ID': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Thyroid_Cancer/gene_data/GSE151180.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Thyroid_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE151180', './preprocessed/Thyroid_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Thyroid_Cancer/GSE151180.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
